In [1]:
import sys
import numpy as np
import os
from image_utils import standardize,distance_map_batch
from IOU_computations import *
from numpy import newaxis
import matplotlib.pyplot as plt
from dataset_generator_2 import Dataset_sat

import logging
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
from resunet_main_fin import predict,criterion
import torch.backends.cudnn as cudnn
import json

/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
!gpustat

cronus  Mon May  7 23:59:48 2018
[0] GeForce GTX TITAN X | 72'C,   0 % |  1898 / 12207 MB | s124262(1887M)
[1] GeForce GTX TITAN X | 77'C,   0 % |  2726 / 12207 MB | s124262(2715M)
[2] GeForce GTX TITAN X | 46'C,   0 % |     0 / 12207 MB |
[3] GeForce GTX TITAN X | 44'C,   0 % |     0 / 12207 MB |
[4] GeForce GTX TITAN X | 44'C,   0 % |     0 / 12207 MB |
[5] GeForce GTX TITAN X | 41'C,   0 % |     0 / 12207 MB |
[6] GeForce GTX TITAN X | 41'C,   2 % |     0 / 12207 MB |
[7] GeForce GTX TITAN X | 77'C,   5 % |  2726 / 12207 MB | s124262(2715M)


In [3]:
INPUT_CHANNELS=9
NB_CLASSES=2


###############
DEFAULT_LAYERS=3
DEFAULT_FEATURES_ROOT=32
BATCH_NORM=True

#################
DEFAULT_FILTER_WIDTH=3
DEFAULT_PADDING=1


################
DISTANCE_NET=False
BINS=15
THRESHOLD=33

###
DEFAULT_N_RESBLOCKS=1
N_DILATED_CONV=3
DEFAULT_GATED=False
DEFAULT_GROUP_NORM=0

####
DEFAULT_HIDDEN_FEATURES=[32,64,128]
DEFAULT_N_RESBLOCKS=1
N_DILATED_CONV=3
DEFAULT_GATED=False
DEFAULT_GROUP_NORM=0
DEFAULT_PADDING=1

###
DEFAULT_HIDDEN_FEATURES_DILATED=[32,64,128]

WEIGHTS_INIT=False

In [4]:
def generate_dataset(path_validation,validation_batch_size):
    val_generator=Dataset_sat.from_root_folder(path_validation,NB_CLASSES)
    val_loader = DataLoader(val_generator, batch_size=validation_batch_size,shuffle=False, num_workers=1)
    
    return val_loader

In [5]:
def predict_pytorch_val(val_loader,restore_path,prediction_path,file_results,distance_net=None,unet_v=2, model_name=''):
    TMP_IOU=prediction_path+'TMP_IOU/'
    if not os.path.exists(TMP_IOU):
            os.makedirs(TMP_IOU)
    loss_fn=nn.CrossEntropyLoss()
    if unet_v==0:
        model=UNet(INPUT_CHANNELS,NB_CLASSES,depth =DEFAULT_LAYERS,n_features_zero =DEFAULT_FEATURES_ROOT,dropout=0,distance_net=DISTANCE_NET,bins=BINS,batch_norm=BATCH_NORM)
    elif unet_v==1 or unet_v==2 or unet_v==4:
        model=UNet(INPUT_CHANNELS,NB_CLASSES,depth =DEFAULT_LAYERS,n_features_zero =DEFAULT_FEATURES_ROOT,width_kernel=DEFAULT_FILTER_WIDTH,dropout=0,distance_net=distance_net,bins=BINS,batch_norm=BATCH_NORM)
    elif unet_v==3:
        model=UNet(INPUT_CHANNELS,NB_CLASSES,num_hidden_features =DEFAULT_HIDDEN_FEATURES,n_resblocks=DEFAULT_N_RESBLOCKS,num_dilated_convs=N_DILATED_CONV,dropout=0,gated=DEFAULT_GATED,kernel_size=DEFAULT_FILTER_WIDTH,padding=DEFAULT_PADDING, group_norm=DEFAULT_GROUP_NORM)
    elif unet_v==5:
        model=DilatedNetwork( INPUT_CHANNELS,NB_CLASSES,num_hidden_features=DEFAULT_HIDDEN_FEATURES_DILATED,n_resblocks=DEFAULT_N_RESBLOCKS, dropout=0, padding=DEFAULT_PADDING, kernel_size=DEFAULT_FILTER_WIDTH,batch_norm=BATCH_NORM)
    if WEIGHTS_INIT:
        model.apply(weights_init)

    model.cuda()
    cudnn.benchmark = True
    model.load_state_dict(torch.load(restore_path))
    


    loss_v=0
    iou_v=0
    iou_acc_v=0
    f1_v=0

    for i_batch,sample in enumerate(val_loader):
        batch_x=standardize(sample['input'])
        batch_y=sample['groundtruth']

        X = Variable(batch_x.float())
        X=X.permute(0,3,1,2).cuda()  
        Y = Variable(batch_y.float())
        Y=Y.cuda()
        if distance_net:
            y_dist=distance_map_batch(batch_y,THRESHOLD,BINS)
            Y_dist = Variable(y_dist.float())
            Y_dist=Y_dist.cuda()
            probs_dist,probs_seg=predict(model,X,distance_net)
            loss_seg=criterion(Y,probs_seg)
            loss_dist=criterion(Y_dist,probs_dist)
            loss=loss_seg+loss_dist
            prediction_seg_v=probs_seg.data.cpu().numpy()
            groundtruth_seg_v=np.asarray(batch_y)
            prediction_dist_v=probs_dist.data.cpu().numpy()
            groundtruth_dist=np.asarray(y_dist)

        else:
            probs=predict(model,X)
            loss=criterion(Y,probs)
            prediction_seg_v=probs.data.cpu().numpy()
            groundtruth_seg_v=np.asarray(batch_y)
            prediction_dist_v=None
            groundtruth_dist=None

        loss_v+=loss.data[0]


        iou_acc,f1,iou=predict_score_batch(TMP_IOU,np.argmax(groundtruth_seg_v,3),np.argmax(prediction_seg_v,3))
        iou_acc_v+=iou_acc
        iou_v+=iou
        f1_v+=f1

#         if i_batch==random_batch_display:
#             batch_x=np.asarray(batch_x)
#             if batch_x.shape[-1]>3:
#                 pansharp=np.stack((batch_x[:,:,:,5],batch_x[:,:,:,3],batch_x[:,:,:,2]),axis=3)
#             else:
#                 pansharp=batch_x
#             plot_summary(prediction_seg_v,groundtruth_seg_v,prediction_dist_v,groundtruth_dist,pansharp,name,self.prediction_path,save_patches)


    loss_v/=val_loader.__len__()
    iou_v/=val_loader.__len__()  
    iou_acc_v/=val_loader.__len__()  
    f1_v/=val_loader.__len__()  
    logging.info("Model {:s}: Verif loss = {:.4f}, Verification  IOU = {:.4f}, IOU Precision = {:.4f}%, F1 IOU= {:.4f}%".format(model_name,loss_v,iou_v,iou_acc_v,f1_v))

    file_results.write(json.dumps({'name': model_name, 'loss':loss_v, 'IOU':iou_v, 'IOU accuracy':iou_acc_v, 'F1 IOU': f1_v}))
    file_results.flush()
        

In [5]:
batch_size=32

root_folder ='/scratch/SPACENET_DATA_PROCESSED/DATASET/120_x_120_8_bands_pansh/'
validation_loader=generate_dataset(root_folder+'TEST/',batch_size)

prediction_path='TEST_SET_RESULTS/'
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)
file_results=open(prediction_path+'models_transfer_learning.txt','w')


## Dilated meli
from unet_dilated_meli import DilatedNetwork
predict_pytorch_val(validation_loader,'MODEL_SPACENET_DILATED_MELI/RESUNET_spacenet_dilated_meliCP84000.pth',prediction_path,file_results,distance_net=False,unet_v=5,model_name='DILATED MELI')

## UNET gated dilated
from unet_val_gated_dilated import UNet
predict_pytorch_val(validation_loader,'MODEL_SPACENET_DILATE_UNET/RESUNET_spacenet_dilated_unetCP28000.pth',prediction_path,file_results,distance_net=False,unet_v=3,model_name='DILATED UNET BEST LOSS')
## UNET gated dilated
from unet_val_gated_dilated import UNet
predict_pytorch_val(validation_loader,'MODEL_SPACENET_DILATE_UNET/RESUNET_spacenet_dilated_unetCP284000.pth',prediction_path,file_results,distance_net=False,unet_v=3,model_name='DILATED UNET BEST IOU METRICS')
## UNET NO dist
from unet_val_2 import UNet
predict_pytorch_val(validation_loader,'MODEL_SPACENET_NODIST5/RESUNET_spacenet_nodist_5CP284000.pth',prediction_path,file_results,distance_net=False,unet_v=2,model_name='RESUNET NO DIST')
## UNET Dist
from unet_val_2 import UNet
predict_pytorch_val(validation_loader,'MODEL_SPACENET_DIST5/RESUNET_spacenet_dist_5CP56000.pth',prediction_path,file_results,distance_net=True,unet_v=2,model_name='RESUNET DIST BEST LOSS')
## UNET Dist
from unet_val_2 import UNet
predict_pytorch_val(validation_loader,'MODEL_SPACENET_DIST5/RESUNET_spacenet_dist_5CP256000.pth',prediction_path,file_results,distance_net=True,unet_v=2,model_name='RESUNET DIST BEST IOU METRICS')

NameError: name 'predict_pytorch_val' is not defined

In [5]:

root_folder ='/scratch/SPACENET_DATA_PROCESSED/DATASET/120_x_120_8_bands_pansh/'
validation_loader=generate_dataset(root_folder+'TEST/',1)
for i_batch,sample in enumerate(validation_loader):
    print(i_batch)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

TypeError: Traceback (most recent call last):
  File "/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 132, in default_collate
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 132, in <dictcomp>
    return {key: default_collate([d[key] for d in batch]) for key in batch[0]}
  File "/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 119, in default_collate
    raise TypeError(error_msg.format(elem.dtype))
TypeError: batch must contain tensors, numbers, dicts or lists; found object


In [ ]:
val_generator=Dataset_sat.from_root_folder(root_folder+'TEST/',NB_CLASSES)
val_generator.__getitem__(1506)